In [13]:
import sys
import os
import sqlite3
import ebisu
from memorizesrs import schedule
from datetime import datetime, timedelta
from math import inf


COLLECTION_PATH = "/Users/jacob/Library/Application Support/Anki2/Minerva/collection.anki2"
ANKI_PATH = '/Users/jacob/anki'

os.chdir(ANKI_PATH)
from anki import Collection

col = Collection(COLLECTION_PATH)

reviewResults = col.db.list("SELECT ease FROM revlog WHERE cid = 1286321546300") 
reviewResults = [ease != 1 for ease in reviewResults]

reviewTimes = col.db.list("SELECT id FROM revlog WHERE cid = 1286321546300")
reviewTimes = [datetime.fromtimestamp(time / 1000.00) for time in reviewTimes]


col.close()

# most commonly reviewed card
# df = pd.read_sql_query("SELECT * FROM revlog WHERE cid = 1286321546300", cnx)

# currentcardid = df.cid.value_counts().idxmax()

In [15]:
learnTime = datetime.fromtimestamp(1286321546300 / 1000.00)

initialModel = ebisu.defaultModel(0.25, 3) # initial half life is 15 minutes, alpha = beta = 3

In [16]:
def historyToEbisuModel(learnTime, reviewTimes, reviewResults, initialModel = ebisu.defaultModel(0.25,3)):
    """Convert history of quizzes to Ebisu model"""
    
    assert len(reviewTimes) == len(reviewResults)

    previousTime = learnTime
    model = initialModel
    
    for (reviewTime, result) in zip(reviewTimes, reviewResults):
        
        model = ebisu.updateRecall(model, result, (reviewTime - previousTime).total_seconds() / 3600)
        previousTime = reviewTime
        
    return model


ebisuModel = historyToEbisuModel(learnTime, reviewTimes, reviewResults)

ebisuModel

(10.812834430355084, 11.795654915767313, 144.61125755257288)

In [17]:
def ebisuModelToMemorizeInterval(model, q=10.0, T=inf):
    """Use an Ebisu model to draw a MEMORIZE interval"""
    return schedule(lambda t: ebisu.predictRecall(initialModel, t, exact=True), q, T)

dueIn = ebisuModelToMemorizeInterval(ebisuModel, q=10.0, T=inf)
print(f"Quiz due in {dueIn} hours")


Quiz due in 1.425266703655391 hours


In [18]:
for model in [initialModel, ebisuModel]:
    print("Ebisu halflife={} hours".format(ebisu.modelToPercentileDecay(model)))
    print(pd.DataFrame([ebisuModelToMemorizeInterval(model) for n in range(10_000)]).describe())

Ebisu halflife=0.25000000000000017 hours
                  0
count  10000.000000
mean       3.557698
std        3.144188
min        0.009033
25%        1.318617
50%        2.652023
75%        4.804148
max       29.914484
Ebisu halflife=135.62746669780623 hours
                  0
count  10000.000000
mean       3.576191
std        3.162954
min        0.018319
25%        1.315564
50%        2.651831
75%        4.868864
max       32.569499
